# Importando Bibliotecas

In [1]:
import xarray as xr #abrir os arquivos .clt do WRF (xarray pro caso do netcdf)
import pandas as pd #abrir planilha merge_wrf
import numpy as np # criar arrays que auxiliam na criação de novas colunas dos DataFrames
import os # criar listas com os nomes dos arquivos
import datetime as dt # Converter as datas dos arquivos em um formato mais "maleável"

In [2]:
# Tirando os Warnings para não poluir a execução dos códigos com mensagens de aviso
from warnings import filterwarnings
filterwarnings('ignore')

# Carregando as pastas com os arquivos

Aqui a gente está "pegando" o nome das pastas que separam os arquivos do WRF por ano para depois acessar o conteúdo de cada uma delas, ou seja, para acessar os arquivos existentes dentro dessas pastas

In [3]:
# Salvando arquivos do WRF (.nc) na variavel arquivos_wrf
diretorio = r'E:\WRF_dados\tratados'
pastas_wrf = [os.path.join(diretorio, pasta) for pasta in os.listdir(diretorio)]

# Ordenando pela grade 1, 2 e 3
pastas_wrf.sort()
pastas_wrf[:5] # mostrando os 5 primeiros

['E:\\WRF_dados\\tratados\\WRF-2016-01-16',
 'E:\\WRF_dados\\tratados\\WRF-2020-12-23',
 'E:\\WRF_dados\\tratados\\WRF-2021-10-28']

# Arquivo do WRF de exemplo para testes

Para entender melhor a estrutura dos arquivos do WRF e como podemos extrair as informações necessárias, abrimos um arquivo qualquer e verificamos como os dados estão distribuidos utilizando a biblioteca xarray (utilizada principalmente para abrir arquivos netCDF)

In [4]:
teste = xr.open_dataset(r"E:\WRF_dados\tratados\WRF-2016-01-16\1_2016-01-15_1_3.nc")
teste

<xarray.Dataset> Size: 12MB
Dimensions:  (time: 73, lev: 19, lat: 46, lon: 46)
Coordinates:
  * time     (time) datetime64[ns] 584B 2016-01-15 ... 2016-01-18
  * lev      (lev) float64 152B 100.0 150.0 200.0 250.0 ... 900.0 950.0 1e+03
  * lat      (lat) float64 368B -29.8 -29.5 -29.21 ... -16.49 -16.17 -15.84
  * lon      (lon) float64 368B -51.6 -51.26 -50.92 ... -37.08 -36.74 -36.4
Data variables:
    chuva    (time, lev, lat, lon) float32 12MB ...

In [5]:
# Verificando em quais índices os horários de ocorrência dos eventos extremos se encontram no arquivo 
teste.time[24:49]

<xarray.DataArray 'time' (time: 25)> Size: 200B
array(['2016-01-16T00:00:00.000000000', '2016-01-16T01:00:00.000000000',
       '2016-01-16T02:00:00.000000000', '2016-01-16T03:00:00.000000000',
       '2016-01-16T04:00:00.000000000', '2016-01-16T05:00:00.000000000',
       '2016-01-16T06:00:00.000000000', '2016-01-16T07:00:00.000000000',
       '2016-01-16T08:00:00.000000000', '2016-01-16T09:00:00.000000000',
       '2016-01-16T10:00:00.000000000', '2016-01-16T11:00:00.000000000',
       '2016-01-16T12:00:00.000000000', '2016-01-16T13:00:00.000000000',
       '2016-01-16T14:00:00.000000000', '2016-01-16T15:00:00.000000000',
       '2016-01-16T16:00:00.000000000', '2016-01-16T17:00:00.000000000',
       '2016-01-16T18:00:00.000000000', '2016-01-16T19:00:00.000000000',
       '2016-01-16T20:00:00.000000000', '2016-01-16T21:00:00.000000000',
       '2016-01-16T22:00:00.000000000', '2016-01-16T23:00:00.000000000',
       '2016-01-17T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 200B 2016-01-16 ... 2016-01-17

Pelo fato dos arquivos conterem os dados de precipitação partindo do dia anterior até o dia posterior ao evento (os dados compreendem 24 horas antes e 24 horas depois do evento), os dados do eventos em questão começam no índice 25 e vão até o 48 (25+23)

*Ps:* O horário de inicio do evento é em 1 hora UTC e o final é em 0 utc do dia seguinte

# Abrindo o arquivo csv "Pontos" com as informações de coordenadas do Merge

In [6]:
# Abrindo arquivo de coordenadas
pontos = pd.read_csv(r'C:\IC\Scripts2\pontos.csv', sep=',')

# Criando o DataFrame com todas as simulações do WRF

In [27]:
# criando lista de dataframes para cada pasta de evento
dataframes = []

# Contador para mostrar o progresso
quantidade_de_arquivos = 576 # 192 arquivos por pasta de evento (até o momento 3 pastas)

# quantidade de arquivos lidos
wrf_lido = 0

# acessando cada pasta de evento
for pasta_evento in pastas_wrf:
  
  # pegando o ano da pasta de evento
  dia_evento = os.path.basename(pasta_evento)[4:]

  # Criando um dataframe para armazenar os valores de chuva de cada ano
  dataframe_evento = pd.DataFrame()

  # Arquivos da pasta
  arquivos_wrf = [os.path.join(pasta_evento, arquivo) for arquivo in os.listdir(pasta_evento) if arquivo.endswith('.nc')]

  # acessado cada arquivo da pasta
  for wrf in arquivos_wrf:

    # Pegando o nome da simulação no nome do arquivo
    nome_simulacao = os.path.basename(wrf)[0:1] + os.path.basename(wrf)[12:].replace('.nc', '') # tira a informação da data e tira o '.nc'

    # Abrindo arquivo
    netCDF = xr.open_dataset(wrf)

    # Abrindo a variável chuva
    chuva_acumulada = netCDF['chuva']

    # Desfazendo o acumulado
    chuva_horaria = chuva_acumulada.diff(dim = 'time')
    
    # criando uma lista para armazenar valores da simulação
    valores_da_simulacao = []

    # Pegando o período de horário por índice
    for horario in range(24, 49):

      # loop para pegar os valores de chuva horária para cada ponto referente a cada latitude e longitude
      for i in range(len(pontos)):
        
        # Pegando a latitude e longitude
        lat = pontos['lat'][i]
        lon = pontos['lon'][i]

        # pegar os valores de chuva para um ponto e um período específico
        chuva_por_ponto = chuva_horaria.sel(lat=lat, lon=lon, method='nearest').isel(time=horario, lev = 18).values
        
        # salvando valores na lista
        valores_da_simulacao.append(chuva_por_ponto)

    # fixando um ponto e printando os valores de chuva simulados para cada simulação para cada horário
    for i in range(28):
      valores_da_simulacao[i::28] = np.array(valores_da_simulacao[i::28]).cumsum()

    # Criando uma coluna com o nome da simulação e preenchendo ela (ex: coluna 1_1_1 com os valores de chuva simulados por ela)
    dataframe_evento[nome_simulacao] = valores_da_simulacao

    # quantidade de arquivos lidos
    wrf_lido += 1
  
    # printando o progresso
    print(f'\r{wrf_lido*100/quantidade_de_arquivos:.2f}%',end = '\r')


  # Repetindo as datas 28 vezes pra ficar lado a lado com os 28 valores de latitude
  dataframe_evento.insert(0,'data',np.repeat(dia_evento, len(dataframe_evento)))

  # preenchendo os valores para horário
  dataframe_evento.insert(1,'horario',np.repeat(np.arange(0, 25), len(pontos)) )

  # Pegando os valores de latitude e longitude
  dataframe_evento.insert(2,'latitude',np.tile(pontos['lat'].values, 25) )
  dataframe_evento.insert(3,'longitude',np.tile(pontos['lon'].values, 25) )

  dataframes.append(dataframe_evento)

# Juntando os dataframes de cada evento em um só
dataframe_total = pd.concat(dataframes)

# Resetando o índice das linhas do dataframe para ordená-las corretamente
dataframe_total = dataframe_total.reset_index(drop = True) # drop é para jogar fora o índice antigo da memória
dataframe_total

,data,horario,latitude,longitude,1_1_1,1_1_14,1_1_16,1_1_2,1_1_3,1_1_4,...,3_7_5,3_7_6,3_8_1,3_8_14,3_8_16,3_8_2,3_8_3,3_8_4,3_8_5,3_8_6
0,2016-01-16,0,-22.95,-44.65,0.053589,9.223862,0.121261,2.851385,2.193081,6.953087,...,1.945419,2.066887,4.473000,5.661263,5.657595,6.717384,11.174793,5.411606,3.109451,2.662842
1,2016-01-16,0,-22.85,-44.25,0.051115,3.580442,0.211135,1.378563,0.720869,2.858562,...,4.400694,1.108181,14.012810,9.136961,1.321850,3.284950,6.132926,5.121437,15.105503,4.790451
2,2016-01-16,0,-22.75,-44.25,0.051115,3.580442,0.211135,1.378563,0.720869,2.858562,...,3.513802,4.100180,6.544037,12.097294,0.700738,2.744808,2.101976,15.729950,3.207767,4.873047
3,2016-01-16,0,-22.85,-44.45,0.053589,9.223862,0.121261,2.851385,2.193081,6.953087,...,5.259535,4.458363,7.160961,8.111401,1.421642,1.724838,10.323914,5.811737,16.126057,1.888603
4,2016-01-16,0,-22.75,-44.35,0.053589,9.223862,0.121261,2.851385,2.193081,6.953087,...,3.636221,2.272820,9.971779,9.060081,0.600858,1.640869,1.465084,11.705414,1.981527,9.920815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,2021-10-28,24,-23.05,-44.45,3.044645,5.110583,2.405195,1.084659,1.383622,0.556904,...,2.604949,6.781530,11.696135,32.294563,10.203671,3.148197,4.877701,4.539671,2.717007,6.641279
2096,2021-10-28,24,-23.05,-44.55,3.044645,5.110583,2.405195,1.084659,1.383622,0.556904,...,10.388331,12.232957,15.046010,17.577991,17.981297,11.352683,10.388556,10.195740,3.419788,18.254883
2097,2021-10-28,24,-23.05,-44.65,3.044645,5.110583,2.405195,1.084659,1.383622,0.556904,...,39.322670,34.518372,23.366398,52.114311,32.349693,16.000404,16.715681,23.661081,11.246054,20.713680
2098,2021-10-28,24,-23.15,-44.05,0.292591,0.905154,0.682669,0.703982,0.720321,0.160078,...,4.387332,1.141704,5.764373,1.349619,2.746046,23.668863,1.844453,1.103312,2.175682,1.525050


In [20]:
# importando dataset-merge.csv
merge = pd.read_csv(r'C:\IC\Scripts2\dataset-merge.csv', sep=',')
merge

,data,horario,latitude,longitude,chuva
0,2016-01-16,0,-22.95,-44.65,3.031250
1,2016-01-16,0,-22.85,-44.25,5.359375
2,2016-01-16,0,-22.75,-44.25,4.796875
3,2016-01-16,0,-22.85,-44.45,3.921875
4,2016-01-16,0,-22.75,-44.35,5.000000
...,...,...,...,...,...
2095,2021-10-28,24,-23.05,-44.45,1.609375
2096,2021-10-28,24,-23.05,-44.55,1.625000
2097,2021-10-28,24,-23.05,-44.65,1.734375
2098,2021-10-28,24,-23.15,-44.05,1.781250


In [ ]:
# pegando valores únicos de chuva horaria
datas = merge['data'].unique()

# gambiarra para transformar chuva horária para acumulado
for data in datas:
    for _,row in pontos.iterrows():
        lat = row[0]
        lon = row[1]
        merge.loc[(merge['data'] == data) & (merge['latitude'] == lat) & (merge['longitude'] == lon), 'chuva'] = merge.loc[(merge['data'] == data) & (merge['latitude'] == lat) & (merge['longitude'] == lon), 'chuva'].cumsum()

In [24]:
# salvando em csv separado com o acumulado de chuva horária
merge.to_csv(r'C:\IC\Scripts2\dataset-merge-gambiarra.csv', index = False)

In [25]:
# abrindo o csv com a gambiarra
merge = pd.read_csv(r'C:\IC\Scripts2\dataset-merge-gambiarra.csv', sep=',')

In [28]:
# juntando o dataframe do WRF com o dataframe do merge
juncao = pd.merge(merge, dataframe_total, on=['data','horario','latitude','longitude'], how='left')
juncao

,data,horario,latitude,longitude,chuva,1_1_1,1_1_14,1_1_16,1_1_2,1_1_3,...,3_7_5,3_7_6,3_8_1,3_8_14,3_8_16,3_8_2,3_8_3,3_8_4,3_8_5,3_8_6
0,2016-01-16,0,-22.95,-44.65,3.031250,0.053589,9.223862,0.121261,2.851385,2.193081,...,1.945419,2.066887,4.473000,5.661263,5.657595,6.717384,11.174793,5.411606,3.109451,2.662842
1,2016-01-16,0,-22.85,-44.25,5.359375,0.051115,3.580442,0.211135,1.378563,0.720869,...,4.400694,1.108181,14.012810,9.136961,1.321850,3.284950,6.132926,5.121437,15.105503,4.790451
2,2016-01-16,0,-22.75,-44.25,4.796875,0.051115,3.580442,0.211135,1.378563,0.720869,...,3.513802,4.100180,6.544037,12.097294,0.700738,2.744808,2.101976,15.729950,3.207767,4.873047
3,2016-01-16,0,-22.85,-44.45,3.921875,0.053589,9.223862,0.121261,2.851385,2.193081,...,5.259535,4.458363,7.160961,8.111401,1.421642,1.724838,10.323914,5.811737,16.126057,1.888603
4,2016-01-16,0,-22.75,-44.35,5.000000,0.053589,9.223862,0.121261,2.851385,2.193081,...,3.636221,2.272820,9.971779,9.060081,0.600858,1.640869,1.465084,11.705414,1.981527,9.920815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,2021-10-28,24,-23.05,-44.45,23.843750,3.044645,5.110583,2.405195,1.084659,1.383622,...,2.604949,6.781530,11.696135,32.294563,10.203671,3.148197,4.877701,4.539671,2.717007,6.641279
2096,2021-10-28,24,-23.05,-44.55,23.664062,3.044645,5.110583,2.405195,1.084659,1.383622,...,10.388331,12.232957,15.046010,17.577991,17.981297,11.352683,10.388556,10.195740,3.419788,18.254883
2097,2021-10-28,24,-23.05,-44.65,21.648438,3.044645,5.110583,2.405195,1.084659,1.383622,...,39.322670,34.518372,23.366398,52.114311,32.349693,16.000404,16.715681,23.661081,11.246054,20.713680
2098,2021-10-28,24,-23.15,-44.05,22.507812,0.292591,0.905154,0.682669,0.703982,0.720321,...,4.387332,1.141704,5.764373,1.349619,2.746046,23.668863,1.844453,1.103312,2.175682,1.525050


In [29]:
# salvando a variável 'juncao' como arquivo csv
juncao.to_csv('juncao.csv', index=False)